In [1]:
import dropbox
import os
import scraperfunctions
import scrapersettings
import csv
import re
from bs4 import BeautifulSoup

In [2]:
team_mapping = scraperfunctions.get_team_mappings()

In [11]:
#team_mapping['697']
team = '697'

In [12]:
url = 'https://stats.ncaa.org/player/game_by_game?game_sport_year_ctl_id=' + \
                scrapersettings.year_index + '&org_id=' + team + '&stats_player_seq=-100'

In [15]:
import requests

In [16]:
result = requests.get(url)
c = result.content

In [17]:
team_mainpage_data_soup = BeautifulSoup(c)

In [49]:
schedule_list = []

In [50]:
for link in team_mainpage_data_soup.find_all('a'): # Locate all links in the document
            if "/game/index/" in link.get('href'): # If they contain a URL segment suggesting it is a game...
                if link.get('href') == "/contests/scoreboards" :
                    continue 
                game_link = str(scrapersettings.domain_base + link.get('href')).split("?")[0] # Strip out any URL variables since we don't need them
                try:
                    opponent_id = link.find_previous("td").find("a").get('href').split("/teams/")[1]
                    print(opponent_id)
                except:
                    opponent_id = 0
                opponent_text = link.find_previous("td").find_previous("td").get_text().encode('utf-8').strip()
                if "@" in opponent_text: # Checks if home or away; note: if it's in a neutral site, this distinction may not be accurate (but a neutral site is flagged). Assumes all games against non-D-I/III competition is at home.
                    home_team = opponent_id
                    away_team = team
                    if "<br/>" in str(link.find_previous("a").encode('utf-8').strip()):
                        neutral = "1"
                    else:
                        neutral = "0"
                else:
                    home_team = team
                    away_team = opponent_id
                    neutral = "0"
                date = link.find_previous("td").find_previous("td").find_previous("td").get_text() # Get the date for the game
                game_id = game_link.split("/")[-1] # Get the game ID from the URL (last set of digits)
                schedule_list.append([game_id, home_team, away_team, date, neutral, game_link])

In [51]:
link

<a href="/teams/471234"> Arkansas </a>

In [15]:
roster_page_data = scraperfunctions.grabber(roster_url, scrapersettings.params, scrapersettings.http_header) # Grab the main page for each team
roster_page_data_soup = BeautifulSoup(roster_page_data)
stat_grid = roster_page_data_soup.select('#stat_grid')

In [20]:
hit_headers = []

In [21]:
for row,val in enumerate(roster_page_data_soup.select('th')):
        hit_headers.append(val.get_text())

In [26]:
pitching_roster_url = str(scrapersettings.domain_base) + "/team/" + team + "/stats?id=" + str(scrapersettings.year_index) + "&year_stat_category_id=14644"
pitching_team_name = team_mapping[team][0]
pitching_page_data = scraperfunctions.grabber(pitching_roster_url,scrapersettings.params,scrapersettings.http_header)
pitching_page_data_soup = BeautifulSoup(pitching_page_data)
pitching_stat_grid = pitching_page_data_soup.select('#stat_grid td , .sorting_asc , .sorting')

In [28]:
pitching_stat_grid

[<td>18</td>,
 <td><a href="/player/index?game_sport_year_ctl_id=14781&amp;stats_player_seq=1757427">Janofsky, Brandon</a></td>,
 <td>Sr</td>,
 <td>INF</td>,
 <td>28</td>,
 <td>28</td>,
 <td data-order="28" style="text-align:right">\n<div>\n      \n        \n          28\n        \n      \n    </div>\n</td>,
 <td data-order="-" style="text-align:right">\n<div>\n</div>\n</td>,
 <td data-order="-" style="text-align:right">\n<div>\n</div>\n</td>,
 <td data-order="-" style="text-align:right">\n<div>\n</div>\n</td>,
 <td data-order="-" style="text-align:right">\n<div>\n</div>\n</td>,
 <td data-order="-" style="text-align:right">\n<div>\n</div>\n</td>,
 <td data-order="0" style="text-align:right">\n<div>\n      \n        \n          0\n        \n      \n    </div>\n</td>,
 <td data-order="-" style="text-align:right">\n<div>\n</div>\n</td>,
 <td data-order="-" style="text-align:right">\n<div>\n</div>\n</td>,
 <td data-order="0" style="text-align:right">\n<div>\n      \n        \n          0\n